In [2]:
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filte
rwarnings('ignore')

AttributeError: module 'warnings' has no attribute 'filte'

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [11]:
cands = pd.read_csv('./candidates.txt', header=None)
cands = cands[0].to_list()
print(cands)

uri_re = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
twi_re = re.compile(r"pic.twitter.com/(?:[a-zA-Z]|[0-9])+")

def remove_uri(tweet):
    try:
        tweet = re.sub(uri_re, "", tweet)
        tweet = re.sub(twi_re, "", tweet)
    except TypeError:
#         print('error in tweet content', tweet)
        tweet = ""
    return tweet

# return string
def get_recent_tweets_string(name, n=-1):
    df_1 = pd.read_json(f"./data/{name}.json", lines=True)
    print("%s's %s tweets" % (name, len(df_1['tweet'])))
    return "\n".join(list(map(remove_uri, df_1['tweet'][0:n].to_list())))

def get_recent_tweets_list(name, n=-1):
    df_1 = pd.read_json(f"./data/{name}.json", lines=True)
    print("%s's %s tweets" % (name, len(df_1['tweet'])))
    return list(map(remove_uri, df_1['tweet'][0:n].to_list()))

# ==== tokenizer ====
import unicodedata

def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False


def _is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False


def _is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
            (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

def _clean_text(text):
    """Performs invalid character removal and whitespace cleanup on text."""
    output = []
    for char in text:
        cp = ord(char)
        if cp == 0 or cp == 0xfffd or _is_control(char):
            continue
        if _is_whitespace(char):
            output.append(" ")
        else:
            output.append(char)
    return "".join(output)

def _run_split_on_punc(text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
        char = chars[i]
        if _is_punctuation(char):
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1

    return ["".join(x) for x in output]
    
def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
      return []
    tokens = text.split()
    return tokens

def tokenize(text):
    """Tokenizes a piece of text."""
    text = _clean_text(text)
    orig_tokens = whitespace_tokenize(text)
    split_tokens = []
    for token in orig_tokens:
        token = token.lower()
#       token = self._run_strip_accents(token)
        split_tokens.extend(_run_split_on_punc(token))

    return split_tokens
    
def lexical_diversity(text):
    data = tokenize(text)
    word_count = len(data) or 1
    vocab_size = len(set(data))
    diversity_score = vocab_size / word_count
    return [vocab_size, word_count, diversity_score]

# get_recent_tweets_list(cands[4])[10709]
lex = [lexical_diversity(get_recent_tweets_string(cand)) for cand in cands]
# docs_raw = [get_recent_tweets_string(name, 500) for name in cands]

['andrewYang', 'ewarren', 'BetoORourke', 'BernieSanders', 'KamalaHarris', 'BilldeBlasio', 'JohnDelaney', 'PeteButtigieg', 'Hickenlooper', 'GovernorBullock', 'MichaelBennet', 'amyklobuchar', 'TulsiGabbard', 'JayInslee', 'JulianCastro', 'CoryBooker', 'JoeBiden', 'TimRyan', 'SenGillibrand', 'marwilliamson']
andrewYang's 7545 tweets
ewarren's 4553 tweets
BetoORourke's 5372 tweets
BernieSanders's 10324 tweets
KamalaHarris's 10728 tweets
BilldeBlasio's 1374 tweets
JohnDelaney's 7136 tweets
PeteButtigieg's 6006 tweets
Hickenlooper's 3288 tweets
GovernorBullock's 6484 tweets
MichaelBennet's 1608 tweets
amyklobuchar's 8644 tweets
TulsiGabbard's 6849 tweets
JayInslee's 3593 tweets
JulianCastro's 3837 tweets
CoryBooker's 54161 tweets
JoeBiden's 1826 tweets
TimRyan's 3464 tweets
SenGillibrand's 17052 tweets
marwilliamson's 15993 tweets


In [14]:
for i, l in enumerate(lex):
    print("%s use %s words in %s tokens, complexity(#words/#tokens) is %s \n" % (cands[i], l[0], l[1], l[2]) )

andrewYang use 10831 words in 161361 tokens, complexity(#words/#tokens) is 0.06712278679482651 

ewarren use 9075 words in 156134 tokens, complexity(#words/#tokens) is 0.058123150627025506 

BetoORourke use 10710 words in 167616 tokens, complexity(#words/#tokens) is 0.06389604810996563 

BernieSanders use 10308 words in 270010 tokens, complexity(#words/#tokens) is 0.03817636383837636 

KamalaHarris use 12765 words in 313707 tokens, complexity(#words/#tokens) is 0.04069083571613002 

BilldeBlasio use 4001 words in 39415 tokens, complexity(#words/#tokens) is 0.10150957757199036 

JohnDelaney use 10902 words in 201060 tokens, complexity(#words/#tokens) is 0.054222620113398984 

PeteButtigieg use 11068 words in 140258 tokens, complexity(#words/#tokens) is 0.0789117198305979 

Hickenlooper use 9527 words in 81686 tokens, complexity(#words/#tokens) is 0.1166295326004456 

GovernorBullock use 10697 words in 158768 tokens, complexity(#words/#tokens) is 0.06737503779099063 

MichaelBennet use 4

In [4]:
def plot(name, recent=None):
    docs_raw = get_recent_tweets_list(name, recent)
    tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                    stop_words = 'english',
                                    lowercase = True,
                                    token_pattern = r'\b[a-zA-Z]{3,}\b',
                                    max_df = 0.5, 
                                    min_df = 10)
    dtm_tf = tf_vectorizer.fit_transform(docs_raw)
#     print(dtm_tf.shape)

    tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
    dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
#     print(dtm_tfidf.shape)

    # for TF DTM
    lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
    lda_tf.fit(dtm_tf)
    # for TFIDF DTM
    lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
    lda_tfidf.fit(dtm_tfidf)
    return tf_vectorizer, lda_tf, dtm_tf

In [13]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[16])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

JoeBiden's 1826 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.154350  0.030162       1        1  11.502871
3      0.053025 -0.011920       2        1  11.270889
4     -0.163670 -0.130060       3        1  10.986819
5      0.144891 -0.042102       4        1  10.950552
8     -0.037005  0.133682       5        1  10.173436
1      0.037800  0.201623       6        1   9.566874
2      0.071719  0.127692       7        1   9.538509
9      0.121885 -0.201069       8        1   8.992474
6     -0.072096 -0.167980       9        1   8.578478
0     -0.310899  0.059973      10        1   8.439098, topic_info=    Category        Freq       Term       Total  loglift  logprob
292  Default  313.000000  president  313.000000  30.0000  30.0000
316  Default  124.000000     romney  124.000000  29.0000  29.0000
198  Default  118.000000        joe  118.000000  28.0000  28.0000
25   Default  422.000000      biden  422.000000  27.0000  27.0000
264  Default  114.000000      obama  114.000000  26.0000  26.0000
243  Default  104.000000     middle  104.000000  25.0000  25.0000
52   Default  100.000000      class  100.000000  24.0000  24.0000
421  Default   93.000000       work   93.000000  23.0000  23.0000
168  Default   72.000000     health   72.000000  22.0000  22.0000
418  Default   92.000000      women   92.000000  21.0000  21.0000
39   Default   76.000000       care   76.000000  20.0000  20.0000
396  Default   65.000000       vice   65.000000  19.0000  19.0000
319  Default   49.000000       ryan   49.000000  18.0000  18.0000
427  Default   77.000000      years   77.000000  17.0000  17.0000
74   Default  148.000000    country  148.000000  16.0000  16.0000
402  Default   84.000000       vote   84.000000  15.0000  15.0000
314  Default   44.000000     rights   44.000000  14.0000  14.0000
277  Default  122.000000     people  122.000000  13.0000  13.0000
365  Default   48.000000        tax   48.000000  12.0000  12.0000
284  Default   80.000000       plan   80.000000  11.0000  11.0000
254  Default   72.000000     nation   72.000000  10.0000  10.0000
240  Default   37.000000   medicare   37.000000   9.0000   9.0000
83   Default  114.000000        day  114.000000   8.0000   8.0000
298  Default   46.000000      proud   46.000000   7.0000   7.0000
388  Default   94.000000      trump   94.000000   6.0000   6.0000
4    Default   40.000000     action   40.000000   5.0000   5.0000
55   Default   33.000000    climate   33.000000   4.0000   4.0000
37   Default   81.000000   campaign   81.000000   3.0000   3.0000
3    Default   56.000000        act   56.000000   2.0000   2.0000
384  Default   47.000000    tonight   47.000000   1.0000   1.0000
..       ...         ...        ...         ...      ...      ...
94   Topic10   15.667137  different   17.488402   2.3623  -4.1830
392  Topic10   12.764848      value   14.594405   2.3384  -4.3879
82   Topic10   15.552543       cuts   18.415096   2.3033  -4.1904
268  Topic10   11.511986        old   13.658495   2.3013  -4.4912
136  Topic10   12.627261    florida   15.491779   2.2678  -4.3987
365  Topic10   39.419449        tax   48.774375   2.2593  -3.2603
159  Topic10   11.780604       grow   14.587772   2.2586  -4.4681
124  Topic10   11.557142      facts   14.617105   2.2374  -4.4873
239  Topic10    9.520001   medicaid   12.703684   2.1838  -4.6812
335  Topic10   10.710604     social   14.631730   2.1603  -4.5634
108  Topic10   25.019620  education   35.248186   2.1295  -3.7149
285  Topic10   16.603958   policies   24.251366   2.0935  -4.1250
420  Topic10   11.302329       word   16.598692   2.0880  -4.5096
81   Topic10   13.847582        cut   20.382490   2.0857  -4.3065
153  Topic10    7.892750        gop   11.741846   2.0751  -4.8687
329  Topic10    8.747341        set   14.582757   1.9612  -4.7658
308  Topic10    6.298882     record   11.516370   1.8689  -5.0942
397  Topic10    7.816128      video   14.513534   1.8534  -4.8784
377  Topi

In [14]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[1])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

ewarren's 4553 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.075128 -0.026187       1        1  13.118726
9      0.178057  0.106673       2        1  12.945697
5      0.318324  0.045515       3        1  11.693862
1     -0.090156  0.017074       4        1  10.695667
2     -0.038409 -0.095250       5        1  10.393597
7     -0.145108 -0.138711       6        1  10.191430
8     -0.114789  0.203391       7        1   8.839378
3     -0.136290  0.135215       8        1   7.829621
4     -0.035490 -0.106079       9        1   7.307701
0      0.138989 -0.141641      10        1   6.984321, topic_info=     Category        Freq             Term       Total  loglift  logprob
646   Default  500.000000            masen  500.000000  30.0000  30.0000
439   Default  246.000000            great  246.000000  29.0000  29.0000
640   Default  243.000000           mapoli  243.000000  28.0000  28.0000
986   Default  157.000000          student  157.000000  27.0000  27.0000
254   Default  165.000000             debt  165.000000  26.0000  26.0000
836   Default  293.000000  realdonaldtrump  293.000000  25.0000  25.0000
148   Default  235.000000             care  235.000000  24.0000  24.0000
124   Default  136.000000            brown  136.000000  23.0000  23.0000
437   Default  153.000000       grassroots  153.000000  22.0000  22.0000
370   Default  675.000000            fight  675.000000  21.0000  21.0000
458   Default  140.000000            happy  140.000000  20.0000  20.0000
1018  Default  191.000000            thank  191.000000  19.0000  19.0000
466   Default  150.000000           health  150.000000  18.0000  18.0000
1133  Default  229.000000            women  229.000000  17.0000  17.0000
607   Default  105.000000             loan  105.000000  16.0000  16.0000
1049  Default  205.000000            trump  205.000000  15.0000  15.0000
249   Default  265.000000              day  265.000000  14.0000  14.0000
766   Default  296.000000             plan  296.000000  13.0000  13.0000
1034  Default  183.000000          tonight  183.000000  12.0000  12.0000
42    Default  171.000000        americans  171.000000  11.0000  11.0000
1141  Default  137.000000          workers  137.000000  10.0000  10.0000
369   Default   84.000000            field   84.000000   9.0000   9.0000
495   Default  131.000000            house  131.000000   8.0000   8.0000
770   Default   86.000000          playing   86.000000   7.0000   7.0000
1082  Default  199.000000             vote  199.000000   6.0000   6.0000
987   Default  116.000000         students  116.000000   5.0000   5.0000
666   Default   79.000000           middle   79.000000   4.0000   4.0000
139   Default  177.000000         campaign  177.000000   3.0000   3.0000
593   Default   75.000000            level   75.000000   2.0000   2.0000
182   Default   78.000000            class   78.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
1095  Topic10   17.007207       wallstreet   20.735001   2.4633  -5.2461
110   Topic10   13.642019      bostonglobe   16.758886   2.4557  -5.4666
1076  Topic10   12.760496         virginia   15.792712   2.4483  -5.5334
152   Topic10    8.686743             case   10.828447   2.4411  -5.9180
515   Topic10   11.716587      independent   14.777353   2.4294  -5.6187
724   Topic10   33.328236           opioid   42.631815   2.4153  -4.5733
484   Topic10   37.744962             hold   53.182382   2.3186  -4.4489
1063  Topic10   35.294969           united   52.174352   2.2707  -4.5160
7     Topic10   49.277827      accountable   78.921192   2.1905  -4.1823
972   Topic10   47.182289           states   87.548684   2.0433  -4.2257
320   Topic10   38.945476        elizabeth   69.065010   2.0886  -4.4176
1133  Topic10   89.484849            women  229.805618   1.7183  -3.5857
925   Topic10   26.059935          senator   49.097532   2.0281  -4.8194
854   Topic10   27.90173

In [15]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[2])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

BetoORourke's 5372 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.299469 -0.006345       1        1  18.006749
4      0.148998 -0.020304       2        1  11.883232
2      0.192026  0.094288       3        1  10.729483
3     -0.037362  0.203847       4        1   9.595552
5      0.161717 -0.076646       5        1   9.569730
1      0.132189 -0.172982       6        1   9.163104
7     -0.017960  0.157725       7        1   8.029865
8     -0.215672 -0.076832       8        1   7.842568
0      0.009577  0.054426       9        1   7.629270
6     -0.074043 -0.157177      10        1   7.550447, topic_info=     Category        Freq         Term       Total  loglift  logprob
1129  Default  550.000000         town  550.000000  30.0000  30.0000
471   Default  497.000000         hall  497.000000  29.0000  29.0000
800   Default  490.000000       people  490.000000  28.0000  28.0000
135   Default  301.000000         care  301.000000  27.0000  27.0000
102   Default  314.000000       border  314.000000  26.0000  26.0000
128   Default  279.000000     campaign  279.000000  25.0000  25.0000
489   Default  220.000000       health  220.000000  24.0000  24.0000
1194  Default  160.000000         vote  160.000000  23.0000  23.0000
1108  Default  360.000000       thanks  360.000000  22.0000  22.0000
790   Default  266.000000         paso  266.000000  21.0000  21.0000
1199  Default  148.000000       voting  148.000000  20.0000  20.0000
587   Default  243.000000         join  243.000000  19.0000  19.0000
1121  Default  312.000000        today  312.000000  18.0000  18.0000
599   Default  177.000000         kids  177.000000  17.0000  17.0000
1104  Default  722.000000        texas  722.000000  16.0000  16.0000
459   Default  286.000000     grateful  286.000000  15.0000  15.0000
461   Default  305.000000        great  305.000000  14.0000  14.0000
537   Default  183.000000  immigration  183.000000  13.0000  13.0000
490   Default  119.000000   healthcare  119.000000  12.0000  12.0000
948   Default  178.000000       rights  178.000000  11.0000  11.0000
726   Default  217.000000      morning  217.000000  10.0000  10.0000
191   Default  192.000000  communities  192.000000   9.0000   9.0000
331   Default   95.000000        early   95.000000   8.0000   8.0000
777   Default  104.000000         pacs  104.000000   7.0000   7.0000
349   Default  180.000000          end  180.000000   6.0000   6.0000
958   Default  223.000000       rourke  223.000000   5.0000   5.0000
951   Default  133.000000         road  133.000000   4.0000   4.0000
338   Default  134.000000    education  134.000000   3.0000   3.0000
1177  Default  192.000000     veterans  192.000000   2.0000   2.0000
1028  Default   87.000000      special   87.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
1042  Topic10   25.778698     standing   34.769521   2.2844  -4.9909
1233  Topic10   16.683502      wichita   22.599292   2.2801  -5.4260
782   Topic10   14.431394       parent   19.846463   2.2649  -5.5710
651   Topic10   24.055088        local   33.446877   2.2540  -5.0601
662   Topic10   20.612521         lost   30.706399   2.1850  -5.2145
675   Topic10   46.047288       making   95.246035   1.8568  -4.4107
396   Topic10   55.608222       family  126.379068   1.7626  -4.2221
87    Topic10   47.043598         best  101.504766   1.8145  -4.3893
323   Topic10   39.476309     dreamers   80.845299   1.8667  -4.5647
395   Topic10   56.869190     families  146.030634   1.6405  -4.1997
245   Topic10   24.479328     critical   41.557327   2.0543  -5.0426
749   Topic10   46.245770        night  109.802343   1.7189  -4.4064
459   Topic10   82.667315     grateful  286.962421   1.3390  -3.8256
999   Topic10   40.271152      service   95.754340   1.7174  -4.5448
512   Topic10   33.008826         home   76.951310   1.7372  -4.7436
478   Topic10   22.391930        happy   38.428223   2.0435  -5.13

In [16]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[3])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

BernieSanders's 10324 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.130577  0.085454       1        1  13.499041
7     -0.162035  0.137071       2        1  12.460302
4     -0.054282 -0.110501       3        1  11.716751
9     -0.024326 -0.166541       4        1  10.141538
3     -0.087677 -0.105725       5        1   9.636353
5     -0.081474 -0.035412       6        1   9.591140
6      0.071390 -0.126204       7        1   8.933941
0      0.382737  0.094005       8        1   8.281405
8     -0.089401  0.210630       9        1   8.130398
2      0.175645  0.017222      10        1   7.609131, topic_info=     Category        Freq              Term       Total  loglift  logprob
123   Default  978.000000            bernie  978.000000  30.0000  30.0000
691   Default  693.000000            health  693.000000  29.0000  29.0000
197   Default  633.000000              care  633.000000  28.0000  28.0000
1366  Default  563.000000           sanders  563.000000  27.0000  27.0000
816   Default  527.000000              join  527.000000  26.0000  26.0000
221   Default  491.000000            change  491.000000  25.0000  25.0000
1662  Default  494.000000              vote  494.000000  24.0000  24.0000
1641  Default  407.000000           vermont  407.000000  23.0000  23.0000
1391  Default  325.000000               sen  325.000000  22.0000  22.0000
1679  Default  343.000000              wall  343.000000  21.0000  21.0000
1489  Default  340.000000            street  340.000000  20.0000  20.0000
1477  Default  301.000000            states  301.000000  19.0000  19.0000
1622  Default  291.000000            united  291.000000  18.0000  18.0000
1673  Default  326.000000              wage  326.000000  17.0000  17.0000
1533  Default  307.000000               tax  307.000000  16.0000  16.0000
1339  Default  513.000000             right  513.000000  15.0000  15.0000
243   Default  358.000000             class  358.000000  14.0000  14.0000
1722  Default  420.000000           workers  420.000000  13.0000  13.0000
824   Default  253.000000           justice  253.000000  12.0000  12.0000
247   Default  230.000000           climate  230.000000  11.0000  11.0000
823   Default  452.000000              just  452.000000  10.0000  10.0000
184   Default  653.000000          campaign  653.000000   9.0000   9.0000
1692  Default  182.000000             watch  182.000000   8.0000   8.0000
1122  Default  269.000000               pay  269.000000   7.0000   7.0000
956   Default  274.000000          medicare  274.000000   6.0000   6.0000
1025  Default  362.000000               new  362.000000   5.0000   5.0000
1161  Default  443.000000         political  443.000000   4.0000   4.0000
978   Default  193.000000           minimum  193.000000   3.0000   3.0000
927   Default  205.000000             major  205.000000   2.0000   2.0000
1001  Default  250.000000          movement  250.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
606   Topic10   18.589734            friend   20.653164   2.4706  -5.9322
1027  Topic10   47.471224              news   58.963857   2.3590  -4.9946
572   Topic10   29.366828        filibernie   34.448380   2.4162  -5.4749
514   Topic10   41.753003     environmental   53.958267   2.3194  -5.1230
1547  Topic10  106.842428             thank  182.046175   2.0429  -4.1834
1600  Topic10   74.898360              tune  126.726243   2.0499  -4.5386
331   Topic10   58.762230          criminal   92.204564   2.1253  -4.7813
894   Topic10  154.554238              live  340.780718   1.7851  -3.8142
901   Topic10   32.414270             local   43.382211   2.2844  -5.3762
998   Topic10   36.881317           morning   52.010103   2.2321  -5.2471
246   Topic10   27.009107           clearly   34.364289   2.3350  -5.5586
168   Topic10   56.237387            budget   97.490816   2.0256  -4.8252
656   Topic10   82.469483             great  173.3526

In [17]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[4])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

KamalaHarris's 10728 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.014616 -0.125558       1        1  13.133305
5     -0.162549  0.048664       2        1  11.150562
1      0.108792 -0.119750       3        1  11.066054
8     -0.155475  0.004411       4        1  10.290923
0     -0.033795 -0.107243       5        1  10.248767
7     -0.036039 -0.058740       6        1  10.146578
4      0.203297 -0.102299       7        1   9.188791
2      0.250892  0.233215       8        1   8.846651
3     -0.034754  0.054030       9        1   8.272052
9     -0.125753  0.173270      10        1   7.656318, topic_info=     Category        Freq             Term       Total  loglift  logprob
2030  Default  837.000000            women  837.000000  30.0000  30.0000
825   Default  897.000000           health  897.000000  29.0000  29.0000
234   Default  871.000000             care  871.000000  28.0000  28.0000
794   Default  332.000000              gun  332.000000  27.0000  27.0000
1972  Default  506.000000             vote  506.000000  26.0000  26.0000
1959  Default  291.000000         violence  291.000000  25.0000  25.0000
993   Default  254.000000           kamala  254.000000  24.0000  24.0000
117   Default  238.000000         attorney  238.000000  23.0000  23.0000
748   Default  234.000000          general  234.000000  22.0000  22.0000
279   Default  446.000000         children  446.000000  21.0000  21.0000
1813  Default  218.000000              tax  218.000000  20.0000  20.0000
992   Default  503.000000          justice  503.000000  19.0000  19.0000
813   Default  222.000000           harris  222.000000  18.0000  18.0000
1821  Default  194.000000       teamkamala  194.000000  17.0000  17.0000
660   Default  454.000000         families  454.000000  16.0000  16.0000
1108  Default  616.000000             make  616.000000  15.0000  15.0000
661   Default  312.000000           family  312.000000  14.0000  14.0000
175   Default  282.000000            black  282.000000  13.0000  13.0000
1557  Default  464.000000           rights  464.000000  12.0000  12.0000
1900  Default  435.000000            trump  435.000000  11.0000  11.0000
265   Default  297.000000           change  297.000000  10.0000  10.0000
574   Default  230.000000         election  230.000000   9.0000   9.0000
301   Default  191.000000          climate  191.000000   8.0000   8.0000
220   Default  599.000000       california  599.000000   7.0000   7.0000
429   Default  540.000000              day  540.000000   6.0000   6.0000
405   Default  178.000000         criminal  178.000000   5.0000   5.0000
37    Default  324.000000   administration  324.000000   4.0000   4.0000
774   Default  262.000000       government  262.000000   3.0000   3.0000
806   Default  169.000000            happy  169.000000   2.0000   2.0000
680   Default  758.000000            fight  758.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
357   Topic10   52.445787        consumers   58.381123   2.4624  -5.0132
208   Topic10   66.456543           budget   75.373059   2.4437  -4.7765
1137  Topic10   42.889365         medicaid   48.225220   2.4524  -5.2144
1313  Topic10   32.155580           photos   36.142203   2.4528  -5.5024
295   Topic10  106.800967            class  137.688584   2.3156  -4.3021
374   Topic10   92.634300     corporations  119.359851   2.3162  -4.4444
1151  Topic10   94.874886           middle  125.594199   2.2891  -4.4205
885   Topic10   32.433033             hurt   40.092981   2.3576  -5.4938
2004  Topic10   36.040792          wealthy   46.133179   2.3228  -5.3884
1603  Topic10   31.470696             scam   39.022378   2.3546  -5.5240
166   Topic10   79.853919              big  131.230357   2.0729  -4.5928
1174  Topic10   81.269560            money  138.600684   2.0358  -4.5752
1900  Topic10  172.215079            trump  435.608172   1.6416  -3.8243
1703  Topic10   38.32828

In [18]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[7])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PeteButtigieg's 6006 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.270333  0.034587       1        1  12.024542
7     -0.154227  0.065244       2        1  10.892576
5      0.162876  0.168857       3        1  10.684137
3      0.131347  0.067910       4        1  10.336984
8      0.053303  0.001234       5        1  10.225078
4     -0.162227 -0.001395       6        1   9.727459
2      0.043877  0.002740       7        1   9.448189
0      0.073018  0.014114       8        1   9.346665
9      0.070712 -0.010694       9        1   8.724815
1      0.051655 -0.342597      10        1   8.589556, topic_info=     Category        Freq        Term       Total  loglift  logprob
938   Default  751.000000       south  751.000000  30.0000  30.0000
72    Default  744.000000        bend  744.000000  29.0000  29.0000
575   Default  202.000000        live  202.000000  28.0000  28.0000
228   Default  297.000000         day  297.000000  27.0000  27.0000
1015  Default  398.000000      thanks  398.000000  26.0000  26.0000
671   Default  156.000000       night  156.000000  25.0000  25.0000
615   Default  229.000000       mayor  229.000000  24.0000  24.0000
669   Default  122.000000        news  122.000000  23.0000  23.0000
543   Default  109.000000      latest  109.000000  22.0000  22.0000
526   Default  261.000000        just  261.000000  21.0000  21.0000
263   Default  127.000000     discuss  127.000000  20.0000  20.0000
364   Default  148.000000     forward  148.000000  19.0000  19.0000
939   Default  236.000000   southbend  236.000000  18.0000  18.0000
121   Default  111.000000      center  111.000000  17.0000  17.0000
459   Default  119.000000        hope  119.000000  16.0000  16.0000
397   Default  368.000000        good  368.000000  15.0000  15.0000
999   Default  137.000000        talk  137.000000  14.0000  14.0000
407   Default  391.000000       great  391.000000  13.0000  13.0000
208   Default   77.000000      county   77.000000  12.0000  12.0000
395   Default  105.000000       going  105.000000  11.0000  11.0000
112   Default   91.000000        care   91.000000  10.0000  10.0000
179   Default  300.000000   community  300.000000   9.0000   9.0000
1129  Default   88.000000        west   88.000000   8.0000   8.0000
1029  Default  251.000000        time  251.000000   7.0000   7.0000
434   Default   90.000000      health   90.000000   6.0000   6.0000
886   Default  103.000000      school  103.000000   5.0000   5.0000
584   Default  144.000000     looking  144.000000   4.0000   4.0000
567   Default  254.000000        like  254.000000   3.0000   3.0000
284   Default   68.000000       early   68.000000   2.0000   2.0000
28    Default  126.000000     america  126.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
524   Topic10   19.667981      joseph   20.541864   2.4112  -5.1429
654   Topic10   16.659800       msnbc   17.606113   2.3994  -5.3089
104   Topic10   10.861365        cafe   11.735281   2.3772  -5.7367
39    Topic10    9.882906  appearance   10.756773   2.3699  -5.8311
1059  Topic10   27.815180        tune   30.321902   2.3683  -4.7963
1000  Topic10   34.931488     talking   38.157056   2.3663  -4.5685
229   Topic10   36.937954        days   42.037912   2.3253  -4.5127
955   Topic10   13.658656       stake   15.650813   2.3185  -5.5075
263   Topic10  105.123510     discuss  127.314368   2.2631  -3.4668
264   Topic10   34.595369  discussing   41.999011   2.2607  -4.5782
114   Topic10   12.540660        case   15.625676   2.2347  -5.5929
913   Topic10   25.844137     shortly   32.219740   2.2341  -4.8698
465   Topic10   24.255802        hour   30.310921   2.2318  -4.9333
1045  Topic10   13.657822       treat   17.535234   2.2047  -5.5076
395   Topic10   81.649608       going  105.534216   2.1980  -3.7195
99    Topic10   48.365418    building   64.443600   2.1676  -4.2431
433   Topic10   50.148435     heading   67.43

In [19]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[11])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

amyklobuchar's 8644 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.123171 -0.138832       1        1  12.143188
6     -0.131197  0.167773       2        1  11.906139
1      0.215405  0.040052       3        1  11.106068
8      0.129596  0.158201       4        1  11.032149
0     -0.208161 -0.046705       5        1  10.771115
3     -0.225031 -0.050126       6        1   9.851552
7      0.211403 -0.041595       7        1   9.642446
4      0.047433  0.046486       8        1   8.508569
5     -0.129853  0.091296       9        1   7.526292
2     -0.032766 -0.226549      10        1   7.512481, topic_info=     Category         Freq        Term        Total  loglift  logprob
1770  Default   769.000000      thanks   769.000000  30.0000  30.0000
742   Default   852.000000       great   852.000000  29.0000  29.0000
1902  Default   307.000000        vote   307.000000  28.0000  28.0000
1120  Default   542.000000   minnesota   542.000000  27.0000  27.0000
1889  Default   190.000000     vikings   190.000000  26.0000  26.0000
1549  Default   527.000000      senate   527.000000  25.0000  25.0000
217   Default   201.000000        care   201.000000  24.0000  24.0000
1277  Default   168.000000       photo   168.000000  23.0000  23.0000
1254  Default   245.000000      passed   245.000000  22.0000  22.0000
1967  Default   379.000000       women   379.000000  21.0000  21.0000
796   Default   503.000000        help   503.000000  20.0000  20.0000
769   Default   267.000000       happy   267.000000  19.0000  19.0000
1172  Default   635.000000        need   635.000000  18.0000  18.0000
539   Default   180.000000    election   180.000000  17.0000  17.0000
783   Default   185.000000      health   185.000000  16.0000  16.0000
696   Default   168.000000        game   168.000000  15.0000  15.0000
1953  Default   190.000000         win   190.000000  14.0000  14.0000
335   Default   321.000000    congrats   321.000000  13.0000  13.0000
508   Default   220.000000        drug   220.000000  12.0000  12.0000
153   Default   304.000000  bipartisan   304.000000  11.0000  11.0000
1972  Default   467.000000        work   467.000000  10.0000  10.0000
1792  Default  1067.000000       today  1067.000000   9.0000   9.0000
603   Default   256.000000    families   256.000000   8.0000   8.0000
11    Default   332.000000         act   332.000000   7.0000   7.0000
540   Default   145.000000   elections   145.000000   6.0000   6.0000
604   Default   239.000000      family   239.000000   5.0000   5.0000
989   Default   224.000000         law   224.000000   4.0000   4.0000
415   Default   492.000000         day   492.000000   3.0000   3.0000
1334  Default   137.000000      prices   137.000000   2.0000   2.0000
1252  Default   285.000000        pass   285.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
1249  Topic10    15.020194     partner    17.901732   2.4131  -6.0266
1886  Topic10    41.133851       video    51.602315   2.3619  -5.0192
633   Topic10    49.338533     finally    67.469615   2.2756  -4.8373
255   Topic10    68.944575       check    98.876768   2.2280  -4.5027
226   Topic10    38.483885        case    52.704403   2.2741  -5.0858
654   Topic10    73.171902       floor   115.255842   2.1343  -4.4432
1656  Topic10    27.007276      spring    35.670035   2.3104  -5.4399
953   Topic10    77.663261     justice   135.803992   2.0298  -4.3836
1646  Topic10    51.050465      speech    85.711785   2.0704  -4.8032
63    Topic10    20.553671  ambassador    25.970692   2.3547  -5.7130
143   Topic10    78.817538        best   166.566721   1.8403  -4.3689
104   Topic10    19.247974    athletes    23.947030   2.3702  -5.7786
13    Topic10    52.066811      action   103.807350   1.8986  -4.7835
593   Topic10    44.576041    facebook    92.175283   1.8621  -4.9388
1120  Topic10   125.212447   minnesota   542.981092   1.1215  -3.9060
1549  Topic10   120

In [20]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[7])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PeteButtigieg's 6006 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.270333  0.034587       1        1  12.024542
7     -0.154227  0.065244       2        1  10.892576
5      0.162876  0.168857       3        1  10.684137
3      0.131347  0.067910       4        1  10.336984
8      0.053303  0.001234       5        1  10.225078
4     -0.162227 -0.001395       6        1   9.727459
2      0.043877  0.002740       7        1   9.448189
0      0.073018  0.014114       8        1   9.346665
9      0.070712 -0.010694       9        1   8.724815
1      0.051655 -0.342597      10        1   8.589556, topic_info=     Category        Freq        Term       Total  loglift  logprob
938   Default  751.000000       south  751.000000  30.0000  30.0000
72    Default  744.000000        bend  744.000000  29.0000  29.0000
575   Default  202.000000        live  202.000000  28.0000  28.0000
228   Default  297.000000         day  297.000000  27.0000  27.0000
1015  Default  398.000000      thanks  398.000000  26.0000  26.0000
671   Default  156.000000       night  156.000000  25.0000  25.0000
615   Default  229.000000       mayor  229.000000  24.0000  24.0000
669   Default  122.000000        news  122.000000  23.0000  23.0000
543   Default  109.000000      latest  109.000000  22.0000  22.0000
526   Default  261.000000        just  261.000000  21.0000  21.0000
263   Default  127.000000     discuss  127.000000  20.0000  20.0000
364   Default  148.000000     forward  148.000000  19.0000  19.0000
939   Default  236.000000   southbend  236.000000  18.0000  18.0000
121   Default  111.000000      center  111.000000  17.0000  17.0000
459   Default  119.000000        hope  119.000000  16.0000  16.0000
397   Default  368.000000        good  368.000000  15.0000  15.0000
999   Default  137.000000        talk  137.000000  14.0000  14.0000
407   Default  391.000000       great  391.000000  13.0000  13.0000
208   Default   77.000000      county   77.000000  12.0000  12.0000
395   Default  105.000000       going  105.000000  11.0000  11.0000
112   Default   91.000000        care   91.000000  10.0000  10.0000
179   Default  300.000000   community  300.000000   9.0000   9.0000
1129  Default   88.000000        west   88.000000   8.0000   8.0000
1029  Default  251.000000        time  251.000000   7.0000   7.0000
434   Default   90.000000      health   90.000000   6.0000   6.0000
886   Default  103.000000      school  103.000000   5.0000   5.0000
584   Default  144.000000     looking  144.000000   4.0000   4.0000
567   Default  254.000000        like  254.000000   3.0000   3.0000
284   Default   68.000000       early   68.000000   2.0000   2.0000
28    Default  126.000000     america  126.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
524   Topic10   19.667981      joseph   20.541864   2.4112  -5.1429
654   Topic10   16.659800       msnbc   17.606113   2.3994  -5.3089
104   Topic10   10.861365        cafe   11.735281   2.3772  -5.7367
39    Topic10    9.882906  appearance   10.756773   2.3699  -5.8311
1059  Topic10   27.815180        tune   30.321902   2.3683  -4.7963
1000  Topic10   34.931488     talking   38.157056   2.3663  -4.5685
229   Topic10   36.937954        days   42.037912   2.3253  -4.5127
955   Topic10   13.658656       stake   15.650813   2.3185  -5.5075
263   Topic10  105.123510     discuss  127.314368   2.2631  -3.4668
264   Topic10   34.595369  discussing   41.999011   2.2607  -4.5782
114   Topic10   12.540660        case   15.625676   2.2347  -5.5929
913   Topic10   25.844137     shortly   32.219740   2.2341  -4.8698
465   Topic10   24.255802        hour   30.310921   2.2318  -4.9333
1045  Topic10   13.657822       treat   17.535234   2.2047  -5.5076
395   Topic10   81.649608       going  105.534216   2.1980  -3.7195
99    Topic10   48.365418    building   64.443600   2.1676  -4.2431
433   Topic10   50.148435     heading   67.43

In [21]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[14])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

JulianCastro's 3837 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.202978 -0.012531       1        1  11.974869
3      0.204885  0.066414       2        1  11.843119
1      0.080158 -0.008807       3        1  10.843910
4     -0.088182  0.214161       4        1  10.766774
6     -0.103827 -0.023141       5        1  10.417674
2     -0.200591  0.157014       6        1   9.688169
8     -0.206438 -0.255025       7        1   9.565500
0     -0.083507 -0.007388       8        1   8.773344
5      0.014325 -0.047676       9        1   8.117006
9      0.180199 -0.083022      10        1   8.009635, topic_info=    Category        Freq          Term       Total  loglift  logprob
565  Default  360.000000           san  360.000000  30.0000  30.0000
30   Default  352.000000       antonio  352.000000  29.0000  29.0000
368  Default  350.000000  juliancastro  350.000000  28.0000  28.0000
695  Default  186.000000          vote  186.000000  27.0000  27.0000
281  Default  321.000000         great  321.000000  26.0000  26.0000
648  Default  238.000000        thanks  238.000000  25.0000  25.0000
669  Default  291.000000         trump  291.000000  24.0000  24.0000
423  Default  133.000000         mayor  133.000000  23.0000  23.0000
644  Default  174.000000         texas  174.000000  22.0000  22.0000
627  Default  127.000000       support  127.000000  21.0000  21.0000
291  Default   96.000000         happy   96.000000  20.0000  20.0000
395  Default  139.000000          like  139.000000  19.0000  19.0000
364  Default  118.000000          join  118.000000  18.0000  18.0000
324  Default   91.000000       housing   91.000000  17.0000  17.0000
696  Default   87.000000         voted   87.000000  16.0000  16.0000
230  Default   89.000000      families   89.000000  15.0000  15.0000
739  Default   84.000000           yes   84.000000  14.0000  14.0000
336  Default  106.000000   immigration  106.000000  13.0000  13.0000
661  Default  188.000000       tonight  188.000000  12.0000  12.0000
249  Default  108.000000         folks  108.000000  11.0000  11.0000
457  Default  159.000000           new  159.000000  10.0000  10.0000
536  Default   62.000000         ready   62.000000   9.0000   9.0000
273  Default  158.000000          good  158.000000   8.0000   8.0000
105  Default  125.000000          city  125.000000   7.0000   7.0000
690  Default   61.000000        vision   61.000000   6.0000   6.0000
699  Default   59.000000        voting   59.000000   5.0000   5.0000
193  Default   67.000000         early   67.000000   4.0000   4.0000
658  Default  350.000000         today  350.000000   3.0000   3.0000
255  Default  109.000000       forward  109.000000   2.0000   2.0000
417  Default  137.000000          make  137.000000   1.0000   1.0000
..       ...         ...           ...         ...      ...      ...
429  Topic10    9.900789       meeting   13.832384   2.1901  -5.2290
692  Topic10    7.888459       visited   11.866653   2.1162  -5.4562
395  Topic10   92.646677          like  139.943678   2.1121  -2.9928
613  Topic10   14.332302         start   21.802929   2.1050  -4.8591
93   Topic10   10.936701       central   16.753657   2.0980  -5.1295
717  Topic10    7.118988          wife   10.938790   2.0950  -5.5588
138  Topic10    9.321611        counts   14.868435   2.0576  -5.2893
182  Topic10   21.142029         doesn   33.846430   2.0540  -4.4703
552  Topic10    9.291836          rico   14.932478   2.0501  -5.2925
249  Topic10   66.928745         folks  108.286159   2.0434  -3.3180
158  Topic10   18.896146          days   31.409106   2.0164  -4.5826
419  Topic10   32.301512        making   57.343150   1.9506  -4.0465
268  Topic10   24.453341       getting   43.457816   1.9495  -4.3248
22   Topic10   38.383860       america   83.511547   1.7472  -3.8740
651  Topic10   15.269776         think   25.618074   2.0071  -4.7957
711  Topic10   31.294884           way   78.972559   1.5989  -4.07

In [22]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[15])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

CoryBooker's 54161 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.116589 -0.086222       1        1  12.160771
3     -0.204849  0.113956       2        1  11.915939
7      0.240479  0.239235       3        1  11.428865
8      0.230968 -0.046412       4        1  10.749729
2     -0.048198 -0.111766       5        1  10.171864
5     -0.093492 -0.161670       6        1   9.822615
4      0.023080 -0.053947       7        1   9.007275
0     -0.129733  0.087638       8        1   8.976058
6     -0.138949  0.149460       9        1   8.009818
1      0.004105 -0.130270      10        1   7.757066, topic_info=     Category         Freq        Term        Total  loglift  logprob
4745  Default  6168.000000      thanks  6168.000000  30.0000  30.0000
1025  Default  4017.000000        cory  4017.000000  29.0000  29.0000
4743  Default  5423.000000       thank  5423.000000  28.0000  28.0000
509   Default  3268.000000      booker  3268.000000  27.0000  27.0000
2637  Default  1809.000000        know  1809.000000  26.0000  26.0000
436   Default  1770.000000        best  1770.000000  25.0000  25.0000
2615  Default  1294.000000        kind  1294.000000  24.0000  24.0000
2727  Default  1408.000000         let  1408.000000  23.0000  23.0000
4413  Default  1404.000000       sorry  1404.000000  22.0000  22.0000
2035  Default  3573.000000       great  3573.000000  21.0000  21.0000
5230  Default  1168.000000       words  1168.000000  20.0000  20.0000
3163  Default  3183.000000      newark  3183.000000  19.0000  19.0000
2504  Default  1030.000000      jersey  1030.000000  18.0000  18.0000
2824  Default  2921.000000        love  2921.000000  17.0000  17.0000
3280  Default   915.000000      office   915.000000  16.0000  16.0000
2744  Default   760.000000        life   760.000000  15.0000  15.0000
1371  Default  1975.000000         don  1975.000000  14.0000  14.0000
5076  Default   992.000000        vote   992.000000  13.0000  13.0000
2104  Default   809.000000       happy   809.000000  12.0000  12.0000
3162  Default  1660.000000         new  1660.000000  11.0000  11.0000
1026  Default  2722.000000  corybooker  2722.000000  10.0000  10.0000
2729  Default   710.000000    letsmove   710.000000   9.0000   9.0000
3809  Default   588.000000        read   588.000000   8.0000   8.0000
5210  Default   619.000000        wish   619.000000   7.0000   7.0000
4761  Default   858.000000       think   858.000000   6.0000   6.0000
2166  Default  2231.000000        help  2231.000000   5.0000   5.0000
2954  Default   774.000000     meeting   774.000000   4.0000   4.0000
5121  Default   577.000000       watch   577.000000   3.0000   3.0000
3593  Default   788.000000       power   788.000000   2.0000   2.0000
339   Default  1180.000000     awesome  1180.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
1978  Topic10    58.431566    godevils    67.665799   2.4098  -6.2535
3685  Topic10   185.681559    progress   296.896155   2.0872  -5.0973
2070  Topic10   213.436940         gun   359.915696   2.0340  -4.9580
1286  Topic10    99.374641      devils   134.640787   2.2529  -5.7225
701   Topic10    92.300472   celebrate   125.734336   2.2474  -5.7963
3163  Topic10   956.441791      newark  3183.846170   1.3539  -3.4582
854   Topic10   167.377041      coffee   302.978466   1.9632  -5.2011
4270  Topic10    84.451342     shelter   114.863468   2.2490  -5.8852
2214  Topic10    67.645989     holiday    84.878176   2.3296  -6.1071
2273  Topic10    90.649857       human   138.299102   2.1342  -5.8144
5284  Topic10   131.666865       youth   262.573121   1.8663  -5.4411
3249  Topic10   402.307641         nwk  1767.402907   1.0765  -4.3242
3193  Topic10   211.091828       night   746.171943   1.2939  -4.9691
5231  Topic10   288.761375        work  1729.255366   0.7667  -4.6558
3447  Topic10   252.267396      people  1538.415674   0.7485  -4.7909
4102  Topic10   119

In [23]:
tf_vectorizer, lda_tf, dtm_tf = plot(cands[0])
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

andrewYang's 7545 tweets


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.150288 -0.028620       1        1  13.287848
7      0.180146 -0.002952       2        1  11.134480
1      0.163088 -0.083387       3        1  10.949596
3     -0.146635 -0.286236       4        1  10.017037
8      0.000681  0.110861       5        1  10.013028
5      0.054164  0.091113       6        1   9.510991
6     -0.105042  0.094521       7        1   9.131766
4     -0.073815  0.131552       8        1   8.886445
2      0.088457 -0.043200       9        1   8.726912
0     -0.311333  0.016348      10        1   8.341897, topic_info=     Category         Freq           Term        Total  loglift  logprob
1056  Default  1118.000000         thanks  1118.000000  30.0000  30.0000
1055  Default   665.000000          thank   665.000000  29.0000  29.0000
500   Default   262.000000         income   262.000000  28.0000  28.0000
67    Default   222.000000          basic   222.000000  27.0000  27.0000
422   Default   490.000000          great   490.000000  26.0000  26.0000
1121  Default   199.000000      universal   199.000000  25.0000  25.0000
584   Default   459.000000            let   459.000000  24.0000  24.0000
1208  Default   182.000000            yes   182.000000  23.0000  23.0000
108   Default   230.000000       campaign   230.000000  22.0000  22.0000
614   Default   188.000000           love   188.000000  21.0000  21.0000
705   Default   479.000000            new   479.000000  20.0000  20.0000
806   Default   136.000000      president   136.000000  19.0000  19.0000
623   Default   437.000000           make   437.000000  18.0000  18.0000
699   Default   403.000000           need   403.000000  17.0000  17.0000
46    Default   115.000000        article   115.000000  16.0000  16.0000
975   Default   178.000000           soon   178.000000  15.0000  15.0000
260   Default   124.000000       dividend   124.000000  14.0000  14.0000
382   Default   148.000000        forward   148.000000  13.0000  13.0000
684   Default   126.000000          month   126.000000  12.0000  12.0000
386   Default   113.000000        freedom   113.000000  11.0000  11.0000
88    Default   128.000000           book   128.000000  10.0000  10.0000
818   Default   116.000000       problems   116.000000   9.0000   9.0000
1204  Default   148.000000       yanggang   148.000000   8.0000   8.0000
1061  Default   238.000000          think   238.000000   7.0000   7.0000
535   Default   207.000000           jobs   207.000000   6.0000   6.0000
438   Default    80.000000      hampshire    80.000000   5.0000   5.0000
554   Default   167.000000           know   167.000000   4.0000   4.0000
764   Default   743.000000         people   743.000000   3.0000   3.0000
292   Default   208.000000        economy   208.000000   2.0000   2.0000
179   Default   131.000000       congrats   131.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
1112  Topic10    20.985196        twitter    28.782075   2.1679  -5.1949
1174  Topic10    12.419142           wife    17.703015   2.1294  -5.7194
443   Topic10    46.664672          happy    68.956120   2.0934  -4.3957
220   Topic10     8.224558            dad    12.178565   2.0913  -6.1316
988   Topic10    23.475910         spread    35.453497   2.0716  -5.0827
161   Topic10    17.935703      cleveland    27.089992   2.0715  -5.3519
390   Topic10    53.117086        friends    82.217257   2.0470  -4.2662
318   Topic10    33.540519          event    52.352745   2.0386  -4.7259
1049  Topic10    29.195509           tell    48.731465   1.9716  -4.8647
718   Topic10    22.597926            nyc    36.596330   2.0018  -5.1208
454   Topic10    20.578952           hear    33.689864   1.9910  -5.2144
1080  Topic10    42.641162        tonight    96.078894   1.6715  -4.4859
709   Topic10    43.623198          night   110.071859   1.5583  -4.4631
1185  Topic10    24.8773